In [1]:
def basis_reconciliation(alice, bob):
    """
        alice: {no. : [bit encoded, basis chosen to encode in ]}
        bob  : {no. : [bit after measurement, basis chosen to measure in]}
        
        First check if the length of both lists are the same
            -> if yes, keep only those bits for alice and bob for which
               the basis encoded in and measured in is the same. 
    """
    basis_bit_alice = list(alice.values())
    basis_bit_bob = list(bob.values())
    
    if len(basis_bit_alice) == len(basis_bit_bob):
        raw_key_alice = []
        raw_key_bob = []

        for i in range(len(basis_bit_alice)):
            if basis_bit_alice[i][1] == basis_bit_bob[i][1]:
                raw_key_alice.append(basis_bit_alice[i][0])
                raw_key_bob.append(basis_bit_bob[i][0])

        return raw_key_alice, raw_key_bob
    
    else:
        return None, None

In [2]:
alice = {1:[0,0],2:[0,1],3:[0,0]}
bob = {1:[1,0],2:[0,0],3:[1,0]}

In [3]:
basis_reconciliation(alice, bob)

([0, 0], [1, 1])

In [27]:
import random
import numpy as np
import math

In [28]:
class LinearPolarizer:
    def __init__(self):
        
        self.x = np.array([[1], [0]])
        self.y = np.array([[0], [1]])
    
    def horizontal_vertical(self, bit):
        if bit == 0:
            return self.x
        else:
            return self.y
    
    def diagonal_polarization(self, bit):
        jones = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
        
        if bit == 0:
            return np.dot(jones, self.x)
        else:
            return np.dot(jones, self.y)
    
    def general_polarization(self, angle, basis):
        """
            angle to be in degrees
        """
        angle = (math.pi/180) * (angle)
        jones = np.array([[np.cos(angle), np.sin(angle)], [np.sin(angle), -np.cos(angle)]])
        
        return np.dot(jones, basis)

class PolarizingBeamSplitter:
    def __init__(self):
        pass
    
    def measure(self, vector, basis):
        """
            basis  : basis chosen by bob to measure polarization encoded photon
                        0 -> horizontal/vertical 
                        1 -> diagonal
            vector : Jones vector for polarized photon
            
            returns a dictionary with probabilities of the encoded bit sent by Alice being 0 or 1
        """
        #horizontal-vertical can be clubbed into an identity matrix
        horizontal = np.array([[1, 0], [0, 0]])
        vertical = np.array([[0, 0], [0, 1]])
        plus_minus = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])

        if basis == 0:
            zero = np.dot(horizontal, vector)[0]
            one = np.dot(vertical, vector)[1]
        
        elif basis == 1:
            zero = np.dot(plus_minus, vector)[0]
            one = np.dot(plus_minus, vector)[1]
        else:
            print("here")
            return None
            
        return {0: zero[0]**2, 1: one[0]**2}

In [37]:
class Alice:
    def __init__(self, n):
        self.n = n
        self.alice = {} #{no. : [bit encoded, basis chosen to encode in ]} no. is some unique number
                        #{1:[0,0],2:[0,1],3:[0,0]} Example

    def generate_and_encode(self): 
        """
            Will generate n bits randomly
            For each bit generated, a basis is chosen in which it is encoded
            Dependency for encoding: <class LinearPolarizer>
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.alice mentioned above
        """
        LP = LinearPolarizer()
        while self.n!= 0:
            self.alice[self.n] = [ random.randint(0,1), random.randint(0,1)]
            if self.alice[self.n][1] == 0:
                self.alice[self.n].append(LP.horizontal_vertical(self.alice[self.n][0]))
            else:
                self.alice[self.n].append(LP.diagonal_polarization(self.alice[self.n][0]))
            self.n-=1
        pass
    

class Bob:
    def __init__(self, n):
        self.n = n
        self.bob = {} #{no. : [bit after measurement, basis chosen to measure in]}
                      #{1:[1,0],2:[0,0],3:[1,0]} Example
    
    def choose_basis_and_measure(self, received):
        """
            received : the data received by bob
            Dependency for measurement: <class PolarizingBeamSplitter>
            
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.bob mentioned above
        """
        #self.bob[n][0] is the measured bit
        
        PBS = PolarizingBeamSplitter()
        
        while self.n!= 0:
            self.bob[self.n] = [0, random.randint(0,1), received]
            self.bob[self.n][0] = 0 if (PBS.measure(self.bob[self.n][2], self.bob[self.n][1])[0]>PBS.measure(self.bob[self.n][2], self.bob[self.n][1])[1]) else 1
            self.n-=1
        pass
    
    

In [34]:
a = Alice(5)

In [35]:
a.generate_and_encode()

In [36]:
a.alice

{5: [1,
  0,
  array([[0],
         [1]])],
 4: [0,
  0,
  array([[1],
         [0]])],
 3: [0,
  1,
  array([[0.70710678],
         [0.70710678]])],
 2: [0,
  0,
  array([[1],
         [0]])],
 1: [1,
  0,
  array([[0],
         [1]])]}

In [45]:
b = Bob(1)

In [46]:
b.choose_basis_and_measure(np.array([[1],[0]]))

In [47]:
b.bob

{1: [0,
  0,
  array([[1],
         [0]])]}